# Simulador Interativo de Localização de CD

Este notebook permite comparar os custos de abrir um novo Centro de Distribuição (CD) 
em **Recife (PE)** ou em **Salvador (BA)**.

Ele usa um conjunto de dados pronto (`cidades_ne150k.csv`), contendo:
- População das cidades do Nordeste > 150 mil habitantes
- Distâncias rodoviárias até Recife e Salvador (km)

Você pode ajustar os parâmetros:
- **n_entregas**: número de entregas por mês
- **custo_km**: custo médio por km por entrega (R$/km)

O resultado mostra:
- Custos totais em Recife e Salvador
- Diferença mensal e economia percentual
- Gráfico de barras comparando os custos
- Gráfico economia acumulada (até 24 meses)


In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import ipywidgets as widgets
from ipywidgets import interact

# Carregar dados já processados
df = pd.read_csv("cidades_ne150k.csv")
total_pop = df["populacao"].sum()

# Parâmetros fixos
cd_area_m2 = 10_000
preco_m2_recife   = 20.5
preco_m2_salvador = 29.5
custo_fix_recife   = cd_area_m2 * preco_m2_recife
custo_fix_salvador = cd_area_m2 * preco_m2_salvador
km_intra_urbano = 15


In [3]:
def simular_para_origem(df_cidades, origem_nome, coluna_dist_km, custo_fixo_origem,
                        n_entregas, custo_km, km_intra_urbano):
    df_sim = df_cidades.copy()
    df_sim["entregas"] = n_entregas * (df_sim["populacao"] / total_pop)
    df_sim["dist_ajustada_km"] = np.where(
        df_sim["municipio"].str.lower() == origem_nome.lower(),
        km_intra_urbano,
        df_sim[coluna_dist_km]
    )
    df_sim["custo_logistico"] = df_sim["entregas"] * df_sim["dist_ajustada_km"] * custo_km
    custo_log_total = df_sim["custo_logistico"].sum()
    custo_total = custo_log_total + custo_fixo_origem
    return {"origem": origem_nome, "custo_total": custo_total}


In [4]:
def simulador_interativo(n_entregas, custo_km):
    res_rec = simular_para_origem(df, "Recife", "dist_recife_km", custo_fix_recife,
                                  n_entregas, custo_km, km_intra_urbano)
    res_sal = simular_para_origem(df, "Salvador", "dist_salvador_km", custo_fix_salvador,
                                  n_entregas, custo_km, km_intra_urbano)

    custo_recife = res_rec["custo_total"]
    custo_salvador = res_sal["custo_total"]
    diff_mensal = custo_salvador - custo_recife
    economia_pct = (diff_mensal / custo_salvador) * 100

    print(f"Custo Recife:   R$ {custo_recife:,.0f}")
    print(f"Custo Salvador: R$ {custo_salvador:,.0f}")
    print(f"\nDiferença mensal (Salvador - Recife): R$ {diff_mensal:,.0f}")
    print(f"Economia percentual ao escolher Recife: {economia_pct:.2f}%")

    # Gráfico comparativo e payback
    fig, ax = plt.subplots(1, 2, figsize=(12,4))

    # Barras
    ax[0].bar(["Recife","Salvador"], [custo_recife, custo_salvador], color=["blue","orange"])
    ax[0].set_title("Custo total mensal")
    for i, v in enumerate([custo_recife, custo_salvador]):
        ax[0].text(i, v + v*0.01, f"R$ {v:,.0f}", ha="center", fontsize=9)

    # Payback acumulado
    meses = list(range(1, 25))
    economia_acumulada = [diff_mensal * m for m in meses]
    ax[1].plot(meses, economia_acumulada, marker="o", color="green")
    ax[1].axhline(0, color="black", linestyle="--")
    ax[1].set_title("Economia acumulada (24 meses)")
    ax[1].set_xlabel("Meses")

    plt.tight_layout()
    plt.show()

interact(
    simulador_interativo,
    n_entregas=widgets.IntSlider(min=10000, max=200000, step=10000, value=100000, description="Entregas/mês"),
    custo_km=widgets.FloatSlider(min=0.2, max=2.0, step=0.1, value=0.5, description="Custo R$/km")
);


interactive(children=(IntSlider(value=100000, description='Entregas/mês', max=200000, min=10000, step=10000), …